[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zjelveh/zjelveh.github.io/blob/master/files/cfc/9_merging.ipynb)

**IMPORTANT**: Save your own copy!
1. Click File → Save a copy in Drive
2. Rename it
3. Work in YOUR copy, not the original

---

# 9. Merging DataFrames
## CCJS 418E: Coding for Criminology

**Today's Goals:**
- Understand why we need to merge datasets
- Learn the four types of merges (inner, left, right, outer)
- Practice combining datasets with different structures
- Apply merging to NYPD arrest data to answer real questions

**The Big Question:** How do we combine information from different sources to add context to our analysis?

---

**📝 Connection to Problem Set 3:**

This lecture teaches the merging skills you'll need for **Problem Set 3 (COVID-19 and NYPD Arrests)**. Specifically:
- **Task 3.1** (Calculate arrest rates by borough) → Uses Pattern 1: Aggregate-Merge-Back
- **Task 3.2** (Felony percentage by borough) → Uses Pattern 2: Merge Two Aggregations
- **Task 3.3** (Percentage change by borough) → Uses basic merge with filtered DataFrames
- **Task 5.2** (Which offenses never recovered?) → Uses merge of two aggregations by offense type

Watch for **📝 PS3 Connection** callouts throughout this lecture!

## Setup: Import Libraries

In [ ]:
import pandas as pd
import seaborn as sns

## Part 1: Why Do We Need Merging?

Imagine you're analyzing crime data and you have:
- Dataset 1: Individual arrest records (who, when, where)
- Dataset 2: Borough-level crime statistics (total crimes, crime rates)

**Question:** Which arrests happened in high-crime areas?

To answer this, we need to **merge** (combine) these datasets!

### Real-World Examples of Merging:
- Linking program participation data with arrest outcomes
- Combining demographic data with crime statistics
- Adding neighborhood context to individual incidents

## Part 2: Understanding Merge Basics

### Example: Program Enrollment and Income Data

Let's start with a simple scenario:
- We have people who enrolled in a job training program
- We have their income data
- Each person appears **once** in each dataset

This is called a **one-to-one merge**.

In [ ]:
# Create sample program enrollment data
program_df = pd.DataFrame({
    'person_id': [1, 2, 3, 4],
    'program_completed': ['Yes', 'No', 'Yes', 'Yes']
})

program_df

In [ ]:
# Create sample income data
income_df = pd.DataFrame({
    'person_id': [1, 2, 3, 4],
    'monthly_income': [2500, 1800, 3000, 2700]
})

income_df

### Performing a Merge

The `pd.merge()` function combines two DataFrames based on a common column (the **key**).

**Basic syntax:**
```python
pd.merge(left=df1, right=df2, on='column_name')
```

- `left` and `right`: The two DataFrames to merge
- `on`: The column(s) to match on

In [ ]:
# Merge the datasets on person_id
merged_df = pd.merge(left=program_df, right=income_df, on='person_id')

merged_df

**What happened?**
- Pandas matched rows where `person_id` was the same
- It combined the columns from both DataFrames
- Each person now has both program and income information

### Using the Merged Data

Now we can answer questions that require information from both datasets!

In [ ]:
# Calculate average income by program completion
avg_income = merged_df.groupby('program_completed')['monthly_income'].mean()

avg_income

In [ ]:
# Visualize the comparison
sns.barplot(data=merged_df,
 x='program_completed',
  y='monthly_income', 
  errorbar=None)

## Part 3: One-to-Many Merges

Sometimes one dataset has **multiple rows** for each ID. 

**Example:** We track people's monthly income over time
- Program data: 1 row per person
- Income data: multiple rows per person (one per month)

This is called a **one-to-many merge**.

In [ ]:
# Program data stays the same
program_df2 = pd.DataFrame({
    'person_id': [1, 2, 3],
    'program_completed': ['Yes', 'No', 'Yes']
})

program_df2

In [ ]:
# Income data now has multiple months per person
income_df2 = pd.DataFrame({
    'person_id': [1, 1, 1, 2, 2, 3],
    'month': ['Jan', 'Feb', 'Mar', 'Jan', 'Feb', 'Jan'],
    'monthly_income': [2500, 2600, 2700, 1800, 1900, 3000]
})

income_df2

In [ ]:
# Perform the merge
merged_df2 = pd.merge(left=program_df2, right=income_df2, on='person_id')

merged_df2

**What happened?**
- Person 1 appears 3 times (once for each month)
- Person 2 appears 2 times (once for each month)
- Person 3 appears 1 time (only one month of data)

**Key insight:** The program information gets **repeated** for each income record!

This is normal and expected in one-to-many merges.

## Part 4: Types of Merges

What if some people appear in one dataset but not the other?

There are **4 types of merges** that handle this differently:

1. **Inner merge** (`how='inner'`): Keep only people who appear in **BOTH** datasets
2. **Left merge** (`how='left'`): Keep **ALL** people from left dataset
3. **Right merge** (`how='right'`): Keep **ALL** people from right dataset
4. **Outer merge** (`how='outer'`): Keep **ALL** people from **BOTH** datasets

### Example Scenario: Some Overlap

Let's create data where:
- Some people are in the program but don't have arrest records
- Some people have arrest records but weren't in the program

In [ ]:
# Program data
program_df3 = pd.DataFrame({
    'person_id': [1, 2, 3, 4],
    'program_completed': ['Yes', 'No', 'Yes', 'Yes']
})

program_df3

In [ ]:
# Arrest data - notice some different person_ids
arrest_df = pd.DataFrame({
    'person_id': [1, 2, 5, 6],
    'arrested': ['No', 'Yes', 'Yes', 'No']
})

arrest_df

**Who appears where?**
- Person 1, 2: In BOTH datasets
- Person 3, 4: Only in program data
- Person 5, 6: Only in arrest data

Let's see how different merge types handle this!

### Type 1: Inner Merge (Only Matches)

**Use when:** You only want complete cases where data exists in both datasets

In [ ]:
inner_merge = pd.merge(program_df3, arrest_df, on='person_id', how='inner')

inner_merge

**Result:** Only person 1 and 2 (they appear in both datasets)

### Type 2: Left Merge (Keep All From Left)

**Use when:** You want to keep all records from the first (left) dataset

In [ ]:
left_merge = pd.merge(program_df3, arrest_df, on='person_id', how='left')

left_merge

**Result:** 
- All 4 people from program data
- Person 3 and 4 have `NaN` for arrested (no arrest data for them)

**NaN means:** No matching record in the right dataset

### Type 3: Right Merge (Keep All From Right)

**Use when:** You want to keep all records from the second (right) dataset

In [ ]:
right_merge = pd.merge(program_df3, arrest_df, on='person_id', how='right')

right_merge

**Result:**
- All 4 people from arrest data
- Person 5 and 6 have `NaN` for program_completed (no program data for them)

### Type 4: Outer Merge (Keep All From Both)

**Use when:** You want to keep everyone from both datasets

In [ ]:
outer_merge = pd.merge(program_df3, arrest_df, on='person_id', how='outer')

outer_merge

**Result:**
- All 6 people (1, 2, 3, 4, 5, 6)
- NaN values where data is missing from one dataset

---

**📝 PS3 Connection: Task 3.3**

In Problem Set 3, you'll merge filtered DataFrames for different years:
```python
# Filter to 2019
df_2019 = borough_year_df[borough_year_df['year'] == 2019]

# Filter to 2020
df_2020 = borough_year_df[borough_year_df['year'] == 2020]

# Merge to compare years
change_df = pd.merge(df_2019, df_2020, on='ARREST_BORO', how='inner')
```

You'll use the **inner merge** type because you only want boroughs that appear in BOTH years (which will be all of them). This pattern lets you calculate percentage change: `(2020 - 2019) / 2019 * 100`

In [ ]:
# Visualize merge types comparison
merge_comparison = pd.DataFrame({
    'Merge Type': ['Inner', 'Left', 'Right', 'Outer'],
    'Number of Rows': [len(inner_merge), len(left_merge), len(right_merge), len(outer_merge)]
})

sns.barplot(data=merge_comparison, x='Merge Type', y='Number of Rows', palette='Set2')

## Part 5: Real Data - NYPD Arrests

Now let's apply merging to real data!

We'll use the NYPD arrest data you've seen in Lecture 8 (from the 2014 police pullback study).

We'll learn three merge patterns in order of difficulty:
- **Pattern 1**: Merge Two Aggregations
- **Pattern 2**: Aggregate-Merge-Back
- **Pattern 3**: Compare Filtered DataFrames

In [ ]:
# Load NYPD arrest data
df = pd.read_csv('https://raw.githubusercontent.com/zjelveh/zjelveh.github.io/refs/heads/master/files/cfc/nypd_arrests_2013_2015_garner.csv')

# Convert date to datetime
df['ARREST_DATE'] = pd.to_datetime(df['ARREST_DATE'])

print(f"Loaded {len(df):,} arrest records")
df.head()

## Pattern 1: Merge Two Aggregations

**Question**: In which borough were assault arrests the highest percentage of total arrests in December 2014?

**Why this matters**: During the pullback, overall arrests dropped dramatically. But did some boroughs maintain enforcement for serious crimes (like assault) while reducing other arrests? Assault is a serious crime that's typically reactive (responding to 911 calls).

**The Strategy**:
1. Count **total** arrests by borough in Dec 2014
2. Count **assault** arrests by borough in Dec 2014
3. Merge these two tables together on borough
4. Calculate assault_pct = (assault_arrests / total_arrests) × 100

This pattern merges TWO separate aggregations to create a new comparison metric.

---

📝 **PS3 Connection**: Task 3.2 uses this exact pattern:
- Count **total** arrests by borough and year
- Count **felony** arrests by borough and year
- Merge them together on borough and year
- Calculate: what % of each borough's arrests were felonies

In [ ]:
# Extract year for filtering
df['year'] = df['ARREST_DATE'].dt.year

# Filter to December 2014 (peak pullback month)
dec_2014 = df[(df['year'] == 2014) & (df['ARREST_DATE'].dt.month == 12)]

print(f"Arrests in December 2014: {len(dec_2014):,}")

In [ ]:
# Aggregation 1: Total arrests by borough in Dec 2014
total_by_borough = dec_2014.groupby('ARREST_BORO').size().reset_index(name='total_arrests')

print("Total arrests by borough (Dec 2014):")
total_by_borough

In [ ]:
# Create is_assault indicator column
dec_2014['is_assault'] = dec_2014['OFNS_DESC'].str.contains('ASSAULT')

# Check it
print("Sample of is_assault column:")
dec_2014[['OFNS_DESC', 'is_assault']].head(10)

In [ ]:
# Aggregation 2: Count assault arrests by borough
assault_by_borough = dec_2014[dec_2014['is_assault']].groupby('ARREST_BORO').size().reset_index(name='assault_arrests')

print("Assault arrests by borough (Dec 2014):")
print(assault_by_borough)

**Now we have TWO separate aggregations**:

1. `total_by_borough`: Total arrests for each borough
2. `assault_by_borough`: Assault arrests for each borough

Both are grouped by borough, but they're counting different things.

**Next**: Merge them together so we can calculate assault as a percentage of total.

In [ ]:
# Merge the two aggregations together
merged = total_by_borough.merge(assault_by_borough, on='ARREST_BORO', how='left')

merged

**What happened?**
- We matched rows where `ARREST_BORO` was the same
- Now each borough has both its **total arrests** AND its **assault arrests**
- `how='left'` keeps all boroughs even if they had zero assault arrests

In [ ]:
# Calculate assault percentage
merged['assault_pct'] = (merged['assault_arrests'] / merged['total_arrests']) * 100

# Sort by assault percentage
merged_sorted = merged.sort_values('assault_pct', ascending=False)

print("Boroughs ranked by assault arrest percentage:")
merged_sorted

In [ ]:
# Visualize assault percentages by borough
sns.barplot(data=merged_sorted, x='ARREST_BORO', y='assault_pct')

---

### ✅ Pattern 1 Summary: Merge Two Aggregations

**When to use it**: When you want to compare statistics from different subsets or categories.

**Key steps**:
1. Create first aggregation (e.g., total arrests by group)
2. Create second aggregation (e.g., subset arrests by same group)
3. Merge them together on the grouping column
4. Calculate percentages using both aggregations


**PS3 Tasks using this pattern**:
- Task 3.2: Merge total arrests and felony arrests by borough-year to calculate felony percentages
- Task 5.2: Merge offense counts from different years to calculate recovery rates

## Pattern 2: Aggregate-Merge-Back

**Question**: What percentage of all arrests each month happened in Brooklyn?

**Why this matters**: Brooklyn is NYC's most populous borough. Understanding its share of total enforcement helps us see if arrest patterns were uniform across the city or concentrated in specific areas.

**The Strategy**:
1. Count total arrests per month (one number for each month)
2. Count Brooklyn arrests per month
3. Merge monthly totals BACK to Brooklyn's monthly data
4. Calculate percentage: brooklyn_arrests / total_arrests × 100

This pattern **"adds context"** to the Brooklyn data by attaching citywide totals to each Brooklyn row.

---

📝 **PS3 Connection**: Task 3.1 uses this exact pattern:
- Count total arrests per YEAR (one number per year)
- Count arrests by BOROUGH and YEAR (multiple rows)
- Merge yearly totals back to borough-year data
- Calculate: what % of that year's arrests came from each borough

In [ ]:
# Create year-month period for grouping
df['year_month'] = df['ARREST_DATE'].dt.to_period('M')

print("Sample of year-month periods:")
df[['ARREST_DATE', 'year_month']].head(10)

**Why .to_period('M')?**

This converts dates to monthly periods, so:
- 2013-01-05 → 2013-01
- 2013-01-28 → 2013-01

This lets us group all arrests in January 2013 together, regardless of the specific day.

In [ ]:
# Count total arrests per month (citywide)
monthly_totals = df.groupby('year_month').size().reset_index(name='total_arrests')

print("Monthly arrest totals (first 10 months):")
monthly_totals.head(10)

In [ ]:
# Count Brooklyn arrests per month
brooklyn_only = df[df['ARREST_BORO'] == 'K']
brooklyn_monthly = brooklyn_only.groupby('year_month').size().reset_index(name='brooklyn_arrests')

print("Brooklyn arrests by month (first 10 months):")
brooklyn_monthly.head(10)

**Now we have two separate tables**:

1. `monthly_totals`: Citywide arrests for each month
2. `brooklyn_monthly`: Brooklyn arrests for each month

**Next**: Merge them together so each Brooklyn row gets its month's citywide total.

In [ ]:
# Merge monthly totals BACK to Brooklyn data
brooklyn_with_context = brooklyn_monthly.merge(monthly_totals, on='year_month', how='left')

print("Brooklyn data with citywide context:")
brooklyn_with_context.head(10)

**What happened?**
- Each Brooklyn monthly row now has the citywide total for that month
- The `total_arrests` number is "repeated" for each month
- This gives us the context we need to calculate Brooklyn's share

**Key insight**: We aggregated at the CITY level (broader), then merged back to BOROUGH level (more detailed).

This is the **"merge-back"** pattern!

In [ ]:
# Calculate Brooklyn's percentage of total arrests
brooklyn_with_context['brooklyn_pct'] = (brooklyn_with_context['brooklyn_arrests'] /
                                          brooklyn_with_context['total_arrests']) * 100

brooklyn_with_context.head(10)



#### Visualize


In [ ]:
import matplotlib as plt
plt.pyplot.figure(figsize=(10, 4))

brooklyn_with_context['ym'] = brooklyn_with_context['year_month'].dt.to_timestamp()

sns.lineplot(data=brooklyn_with_context, x='ym', y='brooklyn_pct')


In [ ]:
brooklyn_with_context['year'] = brooklyn_with_context['year_month'].dt.year
brooklyn_with_context['month'] = brooklyn_with_context['year_month'].dt.month

In [ ]:
sns.lineplot(data=brooklyn_with_context, x='month', y='brooklyn_pct', hue='year')


---

### ✅ Pattern 2 Summary: Aggregate-Merge-Back

**When to use it**: When you want to add aggregate statistics as context to more detailed records.

**Key steps**:
1. Create aggregation at broader level (e.g., total per month)
2. Create aggregation at detailed level (e.g., by borough and month)
3. Merge the broader aggregation back to detailed data
4. Calculate ratios/percentages using both levels

**PS3 Task using this pattern**:
- Task 3.1: Merge yearly totals back to borough-year data to calculate arrest rate percentages

## Pattern 3: Compare Filtered DataFrames

**Question**: Did felony arrests drop the same amount as misdemeanor arrests during the 2014 NYPD pullback?

**Why this matters**: If both categories dropped equally, that suggests an across-the-board resource constraint. If they dropped differently, that reveals enforcement priorities - discretionary enforcement (misdemeanors) vs. mandatory responses (felonies).

**The Strategy**:
1. Filter to 2013 (pre-pullback baseline year)
2. Filter to 2015 (post-pullback recovery year)
3. Count each arrest category in each year
4. Merge the two years together on arrest category
5. Calculate percentage change

This pattern requires multiple filtering and aggregation steps, plus handling suffixes.

---

📝 **PS3 Connection**: You'll use this exact pattern in:
- **Task 3.3**: Compare 2019 vs 2020 arrests by borough
- **Task 5.2**: Compare 2019 vs 2021 arrests by offense type

In [ ]:
# Explore arrest categories
print("Arrest categories:")
print(df['LAW_CAT_CD'].value_counts())
print("\nF = Felony, M = Misdemeanor")

In [ ]:
# Count arrests by category and year
arrests_by_severity_year = df.groupby(['LAW_CAT_CD', 'year']).size().reset_index(name='arrests')

print(arrests_by_severity_year)

**What we have**: A table showing felonies and misdemeanors for each year.

**Next step**: Pull out just 2013 and 2015 so we can compare them side-by-side.

In [ ]:
# Filter to 2013 and 2015 only
arrests_2013 = arrests_by_severity_year[arrests_by_severity_year['year'] == 2013]
arrests_2015 = arrests_by_severity_year[arrests_by_severity_year['year'] == 2015]

print("2013 (Pre-pullback):")
print(arrests_2013)
print("\n2015 (Post-pullback):")
print(arrests_2015)

### The Merge

Now we merge 2013 and 2015 data **on the arrest category** (LAW_CAT_CD).

This puts both years' data side-by-side for each category.

In [ ]:
# Merge 2013 and 2015 data
comparison = pd.merge(arrests_2013, arrests_2015,
                      on='LAW_CAT_CD',
                      how='inner')#,
#                      suffixes=('_2013', '_2015'))

print(comparison)

**What happened?**
- Pandas matched rows where `LAW_CAT_CD` was the same (F or M)
- It added `_2013` and `_2015` suffixes to distinguish the arrest counts and years
- Now each row has both years' data side-by-side
- `how='inner'` keeps only categories that appear in BOTH years (both F and M qualify)

**Key insight**: The suffixes parameter lets us keep track of which year each column came from!

In [ ]:
# Calculate percentage change from 2013 to 2015
comparison['pct_change'] = ((comparison['arrests_2015'] - comparison['arrests_2013']) /
                            comparison['arrests_2013']) * 100

print(comparison)


In [ ]:

print("\nPercentage change from 2013 to 2015:")
for _, row in comparison.iterrows():
    category = "Felonies" if row['LAW_CAT_CD'] == 'F' else "Misdemeanors"
    print(f"{category}: {row['pct_change']:+.1f}%")

In [ ]:
# Visualize the comparison
sns.barplot(data=comparison, x='LAW_CAT_CD', y='pct_change')

**Key finding**: Misdemeanors dropped much more than felonies. This suggests discretionary enforcement (misdemeanors like minor violations) was reduced more than mandatory responses to serious crimes (felonies).

---

### ✅ Pattern 3 Summary: Compare Filtered DataFrames

**When to use it**: When you want to compare the same metric across different time periods or conditions.

**Key steps**:
1. Create aggregation for condition 1 (e.g., 2013)
2. Create aggregation for condition 2 (e.g., 2015)
3. Merge on the grouping column (e.g., LAW_CAT_CD)
4. Use `suffixes=` to distinguish the columns
5. Calculate change/difference

**PS3 Tasks using this pattern**:
- Task 3.3: 2019 vs 2020 by borough
- Task 5.2: 2019 vs 2021 by offense type

## Practice Exercises

Use the NYPD data to answer these questions. Each exercise practices one of the three patterns.

### Exercise 1: Pattern 1 Practice - Drug Enforcement Priority

**Question**: In which borough were drug arrests the highest percentage of total arrests in December 2014?

Use **Pattern 1 (Merge Two Aggregations)**.

**Steps:**
1. Filter to December 2014
2. Count total arrests by borough
3. Filter to drug offenses (OFNS_DESC contains 'DRUG'), count by borough
4. Merge and calculate drug arrest percentage

**Goal**: Determine which borough had the highest percentage of drug arrests.

**Hint**: Drug offenses have 'DRUG' or 'DANGEROUS DRUGS' in OFNS_DESC. Use:
```python
drug_arrests = df[df['OFNS_DESC'].str.contains('DRUG', na=False)]
```

In [ ]:
# Your code here


### Exercise 2: Pattern 2 Practice - Queens' Share Over Time

**Question**: What percentage of misdemeanor arrests happened in Queens each month?

Use **Pattern 2 (Aggregate-Merge-Back)**.

**Steps:**
1. Filter to misdemeanors only (LAW_CAT_CD == 'M')
2. Count total misdemeanor arrests per month
3. Filter to Queens (Q), count Queens misdemeanor arrests per month
4. Merge monthly totals back to Queens data
5. Calculate percentage

**Goal**: Calculate Queens' average share of misdemeanor arrests.

In [ ]:
# Your code here


### Exercise 3: Pattern 3 Practice - Borough Recovery Rates

**Question**: Did the Bronx and Manhattan recover at the same rate from 2014 to 2015?

Use **Pattern 3 (Compare Filtered DataFrames)**.

**Steps:**
1. Filter to just Bronx (B) and Manhattan (M)
2. Count arrests by borough and year
3. Filter to 2014, filter to 2015
4. Merge them on 'ARREST_BORO' with suffixes
5. Calculate percentage change for each borough

**Goal**: Determine which borough saw a bigger percentage increase from 2014 to 2015.

In [ ]:
# Your code here


## Summary

Today you learned three merge patterns:

### Pattern 1: Merge Two Aggregations
- **Use case**: Compare statistics from different subsets
- **Example**: Assault arrest percentages by borough
- **PS3 use**: Task 3.2

### Pattern 2: Aggregate-Merge-Back
- **Use case**: Add aggregate context to detailed records
- **Example**: Brooklyn's share of total arrests each month
- **PS3 use**: Task 3.1

### Pattern 3: Compare Filtered DataFrames
- **Use case**: Compare same metric across different conditions
- **Example**: Felony vs misdemeanor drops during pullback
- **PS3 use**: Tasks 3.3, 5.2

---

## 📝 Problem Set 3 Roadmap

You now have ALL the tools you need for the merging tasks in PS3:

**Task 3.1: Calculate arrest rates by borough**
- Use **Pattern 2** (Aggregate-Merge-Back)
- Aggregate yearly totals, merge back to borough-year data
- Calculate percentages

**Task 3.2: Felony percentage by borough**
- Use **Pattern 1** (Merge-Two-Aggregations)
- Create total arrests table and felony arrests table
- Merge them together, calculate felony percentage

**Task 3.3: Percentage change by borough (2019→2020)**
- Use **Pattern 3** (Compare-Filtered-DataFrames)
- Filter to 2019, filter to 2020
- Merge them, calculate percentage change

**Task 5.2: Which offense types never recovered? (2019 vs 2021)**
- Use **Pattern 3** again
- Same approach as Task 3.3, but comparing offense types instead of boroughs

**Key Success Tips:**
1. Always use `.reset_index()` after `groupby()` before merging
2. Name your columns clearly (use `name=` parameter in `reset_index()`)
3. Check your merge results: does the row count make sense?
4. Use `how='inner'` when comparing years (you want only matching records)